In [1]:
import numpy as np
import pandas as pd

from datetime import datetime

### datetime 对象

In [2]:
dates = [datetime(2022, 12, 2), datetime(2022, 12, 5),
         datetime(2022, 12, 8), datetime(2022, 12, 12),
         datetime(2022, 12, 15), datetime(2022, 12, 20)]

In [3]:
ts = pd.Series(np.arange(6), index=pd.to_datetime(dates))
ts

2022-12-02    0
2022-12-05    1
2022-12-08    2
2022-12-12    3
2022-12-15    4
2022-12-20    5
dtype: int32

In [4]:
ts.index

DatetimeIndex(['2022-12-02', '2022-12-05', '2022-12-08', '2022-12-12',
               '2022-12-15', '2022-12-20'],
              dtype='datetime64[ns]', freq=None)

### pandas 使用 NumPy 的 datetime64 数据类型以纳秒分辨率存储时间戳

In [5]:
ts.index.dtype

dtype('<M8[ns]')

### DatetimeIndex 中的标量值是 pandas Timestamp 对象

In [6]:
ts.index[2]

Timestamp('2022-12-08 00:00:00')

### 与其他 Series 一样，不同索引的时间序列之间的算术运算会在日期上自动对齐

In [7]:
ts[::2]

2022-12-02    0
2022-12-08    2
2022-12-15    4
dtype: int32

In [8]:
ts + ts[::2]

2022-12-02    0.0
2022-12-05    NaN
2022-12-08    4.0
2022-12-12    NaN
2022-12-15    8.0
2022-12-20    NaN
dtype: float64

# 1.索引、选择、子集：

### 当根据标签索引和选择数据时，时间序列的行为与任何其他 Series 一样：

In [9]:
stamp = ts.index[-1]

ts[stamp]

5

### 为方便起见，还可以传递一个可解释为日期的字符串：

In [10]:
ts["2022-12-20"]

5

### 对于较长的时间序列，可以传递一个年份或一个年份和一个月份，来轻松选择数据切片：

In [11]:
# pandas.date_range 用于生成日期范围
long_ts = pd.Series(np.arange(1000) , index=pd.date_range("2021-12-20" , periods=1000))
long_ts

2021-12-20      0
2021-12-21      1
2021-12-22      2
2021-12-23      3
2021-12-24      4
             ... 
2024-09-10    995
2024-09-11    996
2024-09-12    997
2024-09-13    998
2024-09-14    999
Freq: D, Length: 1000, dtype: int32

In [12]:
# 字符串“2021”被解释为年份,并选择该时间段对应的数据
long_ts["2021"]

2021-12-20     0
2021-12-21     1
2021-12-22     2
2021-12-23     3
2021-12-24     4
2021-12-25     5
2021-12-26     6
2021-12-27     7
2021-12-28     8
2021-12-29     9
2021-12-30    10
2021-12-31    11
Freq: D, dtype: int32

In [13]:
# 指定月份
long_ts["2021-12"]

2021-12-20     0
2021-12-21     1
2021-12-22     2
2021-12-23     3
2021-12-24     4
2021-12-25     5
2021-12-26     6
2021-12-27     7
2021-12-28     8
2021-12-29     9
2021-12-30    10
2021-12-31    11
Freq: D, dtype: int32

### 因为大多数时间序列数据是按时间顺序排列的，所以可以使用时间序列中不包含的时间戳进行切片，以执行范围查询：

In [14]:
# long_ts是从"2021-12-20"开始
long_ts["2021-12-1":"2021-12-30"]

2021-12-20     0
2021-12-21     1
2021-12-22     2
2021-12-23     3
2021-12-24     4
2021-12-25     5
2021-12-26     6
2021-12-27     7
2021-12-28     8
2021-12-29     9
2021-12-30    10
Freq: D, dtype: int32

### 使用 datetime 对象进行切片也可以：

In [15]:
long_ts[datetime(2022,12,1):datetime(2022,12,10)]

2022-12-01    346
2022-12-02    347
2022-12-03    348
2022-12-04    349
2022-12-05    350
2022-12-06    351
2022-12-07    352
2022-12-08    353
2022-12-09    354
2022-12-10    355
Freq: D, dtype: int32

### 和之前一样，可以传递字符串日期、datetime或时间戳。需要注意的是，以这种方式切片会生成原时间序列的视图，就像切片 NumPy 数组一样。这意味着没有数据被复制，切片上的修改将反映在原始数据中。

### 有一个等效的实例方法 truncate，它在两个日期之间对 Series 进行切片：

In [16]:
long_ts.truncate(before="2022-12-5" , after="2022-12-10")

2022-12-05    350
2022-12-06    351
2022-12-07    352
2022-12-08    353
2022-12-09    354
2022-12-10    355
Freq: D, dtype: int32

### 所有这一切也适用于 DataFrame，在其行上建立索引：

In [17]:
# freq='W-FRI' 设置时间间隔为每周五
dates = pd.date_range("2022-01-01" , periods=100 , freq='W-FRI')

In [18]:
df = pd.DataFrame(np.arange(300).reshape(100,3) , columns=["X","Y","Z"] , index=dates)
df

,X,Y,Z
2022-01-07,0,1,2
2022-01-14,3,4,5
2022-01-21,6,7,8
2022-01-28,9,10,11
2022-02-04,12,13,14
...,...,...,...
2023-11-03,285,286,287
2023-11-10,288,289,290
2023-11-17,291,292,293
2023-11-24,294,295,296


In [19]:
# 使用loc索引
df.loc["2022-01",["X","Z"]]

,X,Z
2022-01-07,0,2
2022-01-14,3,5
2022-01-21,6,8
2022-01-28,9,11


# 2.含有重复索引的时间序列：

### 在某些应用中，可能有多个数据观测值落在特定时间戳上。例如：

In [20]:
dates = pd.DatetimeIndex(["2022-01-01", "2022-01-02", "2022-01-02","2022-01-02", "2022-01-03"])

In [21]:
dup_ts = pd.Series(np.arange(5) , index=dates)
dup_ts

2022-01-01    0
2022-01-02    1
2022-01-02    2
2022-01-02    3
2022-01-03    4
dtype: int32

### 可以通过检查其 is_unique 属性来判断索引不是唯一的：

In [22]:
dup_ts.index.is_unique

False

### 对该时间序列的索引结果是标量值或切片具体取决于时间戳是否重复：

In [23]:
# 唯一值
dup_ts["2022-01-01"]

0

In [24]:
# 重复值
dup_ts["2022-01-02"]

2022-01-02    1
2022-01-02    2
2022-01-02    3
dtype: int32

### 假设您想要聚合具有非唯一时间戳的数据。一种方法是使用 groupby 并传递 level=0 （唯一的级别）：

In [25]:
grouoped = dup_ts.groupby(level=0)

In [26]:
grouoped.count()

2022-01-01    1
2022-01-02    3
2022-01-03    1
dtype: int64